TIME OF CONFERENCE:   
June 11 at 9 a.m. PT/12 p.m. ET

# Autoreload 2

In [1]:
%load_ext autoreload
%autoreload 2

# Import Relevant Libraries

In [28]:
import pandas as pd
import datetime
from datetime import timezone
import json
import numpy as np
from pandas.io.json import json_normalize
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from math import pi
import time 
import pickle

from nintendo.nintendo_functions_2 import (
    fetch_from_cache_or_api, 
    get_token, 
    get_tweets, 
    store_many_tweets, 
    store_response_tweets, 
    tweet_coll)

from nintendo.trend_radar_functions import (
    reset_index, 
    json_to_df, 
    combine_2_dfs,
    add_time_to_df,
    unique_seconds_list, 
    second_groupings, 
    seconds_dict, 
    unique_words_list,
    vectorize_to_df,
    words_df,
    trend_line,
    drop_time_from_df,
    create_dictionary_for_specified_time,
    top_5_dict_to_df,
    radar_plot_creator,
    completed_words_df,
    trend_function,
    radar_function,
    )  

# Pulls 100 Tweets at a Time (no repeats)

In [26]:
#store_many_tweets(limit=20)

# View Pulled Tweets in Mongo DB

In [ ]:
# DO NOT RUN EVER
#######tweet_coll.delete_many({})

In [29]:
tweet_coll.count_documents({})

10269

### Assign tweet json dicts to variable "tweets"

In [30]:
tweets = list(tweet_coll.find())

WORK ON AUTOMATING WITH FUNCTIONS... AFTER WEBSITE

# Experimental

In [7]:
tweet_df = pd.DataFrame(tweets)

In [8]:
print(tweet_df['created_at'].min())
print(tweet_df['created_at'].max())


Tue Jun 11 16:07:00 +0000 2019
Wed Jun 12 17:04:58 +0000 2019


In [9]:
tweet_df['text']

0        RT @sandraghart: Needed to do this because she...
1        RT @NVIDIAGeForce: We're kicking off E3 Show D...
2        RT @MrGenzoman: #Nessa sketch/color warmup fro...
3        RT @sandraghart: Needed to do this because she...
4        RT @MrGenzoman: #Nessa sketch/color warmup fro...
5        RT @sandraghart: Needed to do this because she...
6        RT @hiraharas: same energy #e3 #NintendoDirect...
7        RT @hiraharas: same energy #e3 #NintendoDirect...
8        RT @MrGenzoman: #Nessa sketch/color warmup fro...
9        RT @sandraghart: Needed to do this because she...
10       RT @Inknettl: Thank you #NintendoDirect https:...
11       RT @hntrluc: Zelda is looking so cute with her...
12       RT @Inknettl: Thank you #NintendoDirect https:...
13       RT @hntrluc: Zelda is looking so cute with her...
14       RT @hiraharas: same energy #e3 #NintendoDirect...
15       RT @MrGenzoman: #Nessa sketch/color warmup fro...
16       RT @hiraharas: same energy #e3 #NintendoDirect.

In [10]:
tweet_df.iloc[-1]['created_at']

'Tue Jun 11 16:32:00 +0000 2019'

In [11]:
tweet_df['timestamp'] = tweet_df['created_at'].apply(pd.Timestamp)

In [24]:
tweet_df.loc[tweet_df['timestamp'] == 'Tue Jun 11 16:33:25 +0000 2019']

,_id,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,...,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,truncated,user,timestamp
9599,5d20eeaa6055bb11b1b217b5,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'e32019', 'indices': [7...",NaN,NaN,0,False,...,NaN,0,0,False,NaN,"<a href=""https://about.twitter.com/products/tw...",I'm more hyped for Marvel Ultimate Alliance th...,False,"{'id': 22874597, 'id_str': '22874597', 'name':...",2019-06-11 16:33:25+00:00
9600,5d20eeaa6055bb11b1b217b6,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'NintendoDirect', 'indi...",NaN,NaN,0,False,...,NaN,0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Eggman gettin it XD #NintendoDirect,False,"{'id': 255566561, 'id_str': '255566561', 'name...",2019-06-11 16:33:25+00:00
9601,5d20eeaa6055bb11b1b217b7,None,None,Tue Jun 11 16:33:25 +0000 2019,"[0, 92]","{'hashtags': [{'text': 'Cadenceofhyrule', 'ind...","{'media': [{'id': 1138484385215320065, 'id_str...",NaN,0,False,...,NaN,0,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#Cadenceofhyrule in Two days! That's all I nee...,False,"{'id': 1855266734, 'id_str': '1855266734', 'na...",2019-06-11 16:33:25+00:00
9602,5d20eeaa6055bb11b1b217b8,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'NintendoDirect', 'indi...","{'media': [{'id': 1138484087142965248, 'id_str...",NaN,0,False,...,NaN,0,0,False,{'created_at': 'Tue Jun 11 16:32:35 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @Nibellion: Cadence Of Hyrule dropping June...,False,"{'id': 392139718, 'id_str': '392139718', 'name...",2019-06-11 16:33:25+00:00
9603,5d20eeaa6055bb11b1b217b9,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'Marvel', 'indices': [0...",NaN,NaN,0,False,...,NaN,0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","#Marvel on #Switch looks really good, not gonn...",False,"{'id': 1129111809003270144, 'id_str': '1129111...",2019-06-11 16:33:25+00:00
9604,5d20eeab6055bb11b1b217ba,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'NintendoE3', 'indices'...",NaN,NaN,0,False,...,NaN,0,0,False,NaN,"<a href=""https://about.twitter.com/products/tw...",Mario and Sonic getting another Olympics game....,False,"{'id': 24643404, 'id_str': '24643404', 'name':...",2019-06-11 16:33:25+00:00
9605,5d20eeab6055bb11b1b217bb,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'NintendoE3', 'indices'...","{'media': [{'id': 1138482239807926272, 'id_str...",NaN,0,False,...,NaN,0,0,False,{'created_at': 'Tue Jun 11 16:25:04 +0000 2019...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @Sam_A_Hartwell: When Animal Crossing hasn'...,False,"{'id': 955965647359528960, 'id_str': '95596564...",2019-06-11 16:33:25+00:00
9606,5d20eeab6055bb11b1b217bc,None,None,Tue Jun 11 16:33:25 +0000 2019,NaN,"{'hashtags': [{'text': 'PassTheController', 'i...",NaN,NaN,0,False,...,NaN,0,0,False,{'created_at': 'Tue Jun 11 16:22:14 +0000 2019...,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @PassController: No More Heroes 3 coming in...,False,"{'id': 102732562, 'id_str': '102732562', 'name...",2019-06-11 16:33:25+00:00
